In [1]:
# 02_Preprocesamiento_y_Seleccion_Caracteristicas.ipynb

# ==============================================================================
# PREPROCESAMIENTO DE DATOS Y SELECCIÓN DE CARACTERÍSTICAS
# ==============================================================================

# Este notebook se enfoca en preparar los datos para el modelado de Machine Learning.
# Incluye la codificación de variables categóricas, el escalado de variables numéricas,
# y la división del dataset en conjuntos de entrenamiento y prueba.

# ------------------------------------------------------------------------------
# 1. Configuración Inicial y Carga de Datos
# ------------------------------------------------------------------------------

# Importar las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import os
import joblib # Para guardar el preprocesador si es necesario

print("--------------------------------------------------")
print("INICIANDO PREPROCESAMIENTO DE DATOS Y SELECCIÓN DE CARACTERÍSTICAS...")
print("--------------------------------------------------")

# Definir la ruta al archivo de datos CSV generado en el EDA
data_path = os.path.join(os.getcwd(), '..', 'data', 'raw', 'simulated_paint_data.csv')

# Verificar si el archivo existe antes de cargarlo
if not os.path.exists(data_path):
    print(f"ERROR: No se encontró el archivo de datos en: {data_path}")
    print("Por favor, asegúrate de haber ejecutado 'python src/data_generator.py' desde la raíz del proyecto.")
else:
    # Cargar el dataset
    df = pd.read_csv(data_path)
    print(f"Datos cargados exitosamente desde: {data_path}")
    print(f"Número de filas inicial: {df.shape[0]}, Número de columnas inicial: {df.shape[1]}")


--------------------------------------------------
INICIANDO PREPROCESAMIENTO DE DATOS Y SELECCIÓN DE CARACTERÍSTICAS...
--------------------------------------------------
Datos cargados exitosamente desde: C:\Users\Víctor\Documents\PaintFormulatorAI\notebooks\..\data\raw\simulated_paint_data.csv
Número de filas inicial: 5000, Número de columnas inicial: 16


In [2]:
# ------------------------------------------------------------------------------
# 2. Identificación de Tipos de Columnas
# ------------------------------------------------------------------------------

print("\n--------------------------------------------------")
print("2. IDENTIFICACIÓN DE TIPOS DE COLUMNAS")
print("--------------------------------------------------")

# Separar la variable objetivo (target) de las características (features)
X = df.drop('exito', axis=1) # Características
y = df['exito']             # Variable objetivo

print(f"Variable objetivo 'y' (exito) separada. X shape: {X.shape}, y shape: {y.shape}")

# Identificar columnas numéricas y categóricas para el preprocesamiento
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

print(f"\nCaracterísticas Numéricas identificadas: {numerical_features}")
print(f"Características Categóricas identificadas: {categorical_features}")

print("\nExplicación para Leonardo (Identificación de Columnas):")
print("Hemos separado los datos en 'X' (las características que describen la formulación y proceso)")
print("y 'y' (el resultado 'Éxito' o 'Falla' que queremos predecir).")
print("Es crucial identificar qué características son numéricas (como porcentajes, temperaturas) y cuáles son categóricas (como tipos o proveedores),")
print("ya que cada tipo requiere un tratamiento diferente antes de que los modelos de Machine Learning puedan entenderlas.")



--------------------------------------------------
2. IDENTIFICACIÓN DE TIPOS DE COLUMNAS
--------------------------------------------------
Variable objetivo 'y' (exito) separada. X shape: (5000, 15), y shape: (5000,)

Características Numéricas identificadas: ['resina_pct', 'pigmento_pct', 'solvente_pct', 'aditivo_pct', 'temperatura_mezcla_C', 'tiempo_mezcla_min', 'viscosidad_cp', 'brillo_unidades', 'poder_cubriente', 'resistencia_abrasion_ciclos', 'estabilidad_almacenamiento_dias']
Características Categóricas identificadas: ['calidad_resina', 'tipo_pigmento', 'tipo_solvente', 'proveedor_aditivo']

Explicación para Leonardo (Identificación de Columnas):
Hemos separado los datos en 'X' (las características que describen la formulación y proceso)
y 'y' (el resultado 'Éxito' o 'Falla' que queremos predecir).
Es crucial identificar qué características son numéricas (como porcentajes, temperaturas) y cuáles son categóricas (como tipos o proveedores),
ya que cada tipo requiere un tratamien

In [3]:
# ------------------------------------------------------------------------------
# 3. Preprocesamiento de Datos (Codificación y Escalado)
# ------------------------------------------------------------------------------

print("\n--------------------------------------------------")
print("3. PREPROCESAMIENTO DE DATOS")
print("--------------------------------------------------")

# Creamos pipelines para transformar las características:
# 1. Para las numéricas: aplicar StandardScaler (estandarización)
# 2. Para las categóricas: aplicar OneHotEncoder (codificación one-hot)

# Paso de preprocesamiento para características numéricas: Estandarización
# StandardScaler ajusta la media a 0 y la desviación estándar a 1.
# Esto es vital para muchos algoritmos que son sensibles a la escala de las características.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Paso de preprocesamiento para características categóricas: One-Hot Encoding
# OneHotEncoder convierte cada categoría en una nueva columna binaria (0 o 1).
# Esto evita que el modelo interprete las categorías como si tuvieran un orden.
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # 'ignore' para nuevas categorías en el futuro
])

# Combinar los preprocesadores usando ColumnTransformer
# Esto permite aplicar diferentes transformaciones a diferentes columnas en paralelo.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

print("\nPreprocesador configurado:")
print("- Columnas numéricas se escalarán con StandardScaler (media=0, desv_est=1).")
print("- Columnas categóricas se codificarán con OneHotEncoder (variables binarias por categoría).")

print("\nExplicación para Leonardo (Preprocesamiento):")
print("Esta es una fase técnica donde preparamos los datos para el 'cerebro' del Machine Learning.")
print("1. 'Escalado de Numéricas': Componentes como 'resina_pct' o 'temperatura_mezcla_C' tienen rangos muy diferentes.")
print("   Al 'escalarlos', nos aseguramos de que ninguno de ellos 'domine' al modelo solo por tener números más grandes.")
print("   Esto ayuda a que el modelo aprenda de manera justa de todas las características.")
print("2. 'Codificación de Categóricas': Las palabras como 'Alta', 'Dióxido de Titanio' o 'ProveedorA' no son números.")
print("   Las convertimos en un formato numérico (0s y 1s) que el modelo pueda procesar, sin implicar un orden falso.")
print("Este 'preprocesador' es como una receta que aplicaremos de forma consistente a todos nuestros datos,")
print("tanto para el entrenamiento como para las futuras predicciones.")


# Ajustar y transformar los datos
X_processed = preprocessor.fit_transform(X)

# Obtener los nombres de las columnas después del One-Hot Encoding
# Esto es útil para entender el DataFrame procesado
onehot_features_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)
processed_feature_names = numerical_features + onehot_features_names.tolist()

# Convertir el array procesado de vuelta a un DataFrame para facilitar la visualización y análisis
X_processed_df = pd.DataFrame(X_processed, columns=processed_feature_names)

print(f"\nDatos procesados. Nueva forma de X: {X_processed_df.shape}")
print("\nPrimeras 5 filas del DataFrame de características procesadas:")
print(X_processed_df.head())

# Guardar el preprocesador ajustado para su uso futuro (en la fase de predicción)
# Esto es vital para asegurar que las nuevas predicciones se preprocesen de la misma manera
preprocessor_path = os.path.join(os.getcwd(), '..', 'data', 'processed', 'preprocessor.joblib')
os.makedirs(os.path.dirname(preprocessor_path), exist_ok=True) # Asegurar que la carpeta 'processed' exista
joblib.dump(preprocessor, preprocessor_path)
print(f"\nPreprocesador guardado exitosamente en: {preprocessor_path}")

print("\nExplicación para Leonardo (Datos Procesados):")
print("Ahora, todas las características de la formulación están en un formato numérico y escalado,")
print("listas para ser 'consumidas' por el algoritmo de Machine Learning. El 'cerebro' del modelo")
print("aprenderá de estos números para identificar patrones que llevan al éxito o al fracaso.")
print("Hemos guardado esta 'receta' de preprocesamiento para que, cuando usted ingrese una nueva fórmula,")
print("la preparemos exactamente de la misma manera antes de pedirle una predicción al modelo.")



--------------------------------------------------
3. PREPROCESAMIENTO DE DATOS
--------------------------------------------------

Preprocesador configurado:
- Columnas numéricas se escalarán con StandardScaler (media=0, desv_est=1).
- Columnas categóricas se codificarán con OneHotEncoder (variables binarias por categoría).

Explicación para Leonardo (Preprocesamiento):
Esta es una fase técnica donde preparamos los datos para el 'cerebro' del Machine Learning.
1. 'Escalado de Numéricas': Componentes como 'resina_pct' o 'temperatura_mezcla_C' tienen rangos muy diferentes.
   Al 'escalarlos', nos aseguramos de que ninguno de ellos 'domine' al modelo solo por tener números más grandes.
   Esto ayuda a que el modelo aprenda de manera justa de todas las características.
2. 'Codificación de Categóricas': Las palabras como 'Alta', 'Dióxido de Titanio' o 'ProveedorA' no son números.
   Las convertimos en un formato numérico (0s y 1s) que el modelo pueda procesar, sin implicar un orden falso.

In [4]:
# ------------------------------------------------------------------------------
# 4. División del Dataset en Entrenamiento y Prueba
# ------------------------------------------------------------------------------

print("\n--------------------------------------------------")
print("4. DIVISIÓN DEL DATASET EN ENTRENAMIENTO Y PRUEBA")
print("--------------------------------------------------")

# Dividir el dataset en conjuntos de entrenamiento y prueba
# test_size=0.2 (20% para prueba, 80% para entrenamiento)
# random_state para reproducibilidad: asegura que la división sea la misma cada vez que se ejecute.
# stratify=y para manejar el desbalanceo: asegura que la proporción de 'exito'/'falla' sea similar
# en ambos conjuntos (entrenamiento y prueba). Esto es CRÍTICO para datasets desbalanceados.
X_train, X_test, y_train, y_test = train_test_split(X_processed_df, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nDataset dividido en entrenamiento y prueba:")
print(f"  X_train shape (características de entrenamiento): {X_train.shape}")
print(f"  X_test shape (características de prueba): {X_test.shape}")
print(f"  y_train shape (variable objetivo de entrenamiento): {y_train.shape}")
print(f"  y_test shape (variable objetivo de prueba): {y_test.shape}")

# Verificar el balance de clases en los conjuntos de entrenamiento y prueba
print("\nBalance de clases en y_train:")
print(y_train.value_counts(normalize=True).round(2) * 100)
print("\nBalance de clases en y_test:")
print(y_test.value_counts(normalize=True).round(2) * 100)

print("\nExplicación para Leonardo (División Entrenamiento/Prueba):")
print("Hemos dividido nuestros datos en dos partes: una para 'enseñar' al modelo (entrenamiento) y otra para 'examinarlo' (prueba).")
print("Imagine que está preparando a un estudiante para un examen: le da material para estudiar (datos de entrenamiento)")
print("y luego lo evalúa con preguntas que nunca ha visto (datos de prueba).")
print("Esto nos permite evaluar qué tan bien el modelo generaliza a nuevas formulaciones, y no solo qué tan bien memoriza las que ya conoce.")
print("La clave aquí es que la proporción de éxitos y fallas es la misma en ambos conjuntos (estratificación),")
print("lo cual es vital por el desbalanceo que observamos en la fase de EDA.")


--------------------------------------------------
4. DIVISIÓN DEL DATASET EN ENTRENAMIENTO Y PRUEBA
--------------------------------------------------

Dataset dividido en entrenamiento y prueba:
  X_train shape (características de entrenamiento): (4000, 25)
  X_test shape (características de prueba): (1000, 25)
  y_train shape (variable objetivo de entrenamiento): (4000,)
  y_test shape (variable objetivo de prueba): (1000,)

Balance de clases en y_train:
exito
1    75.0
0    25.0
Name: proportion, dtype: float64

Balance de clases en y_test:
exito
1    75.0
0    25.0
Name: proportion, dtype: float64

Explicación para Leonardo (División Entrenamiento/Prueba):
Hemos dividido nuestros datos en dos partes: una para 'enseñar' al modelo (entrenamiento) y otra para 'examinarlo' (prueba).
Imagine que está preparando a un estudiante para un examen: le da material para estudiar (datos de entrenamiento)
y luego lo evalúa con preguntas que nunca ha visto (datos de prueba).
Esto nos permite eva

In [5]:
# ------------------------------------------------------------------------------
# 5. Guardar Datos Preprocesados (Opcional, pero buena práctica)
# ------------------------------------------------------------------------------

print("\n--------------------------------------------------")
print("5. GUARDADO DE DATOS PREPROCESADOS")
print("--------------------------------------------------")

# Es una buena práctica guardar los conjuntos preprocesados para que los modelos
# puedan cargarlos directamente sin necesidad de re-ejecutar todo el preprocesamiento
# en notebooks posteriores, lo que ahorra tiempo.

X_train_path = os.path.join(os.getcwd(), '..', 'data', 'processed', 'X_train.csv')
X_test_path = os.path.join(os.getcwd(), '..', 'data', 'processed', 'X_test.csv')
y_train_path = os.path.join(os.getcwd(), '..', 'data', 'processed', 'y_train.csv')
y_test_path = os.path.join(os.getcwd(), '..', 'data', 'processed', 'y_test.csv')

X_train.to_csv(X_train_path, index=False)
X_test.to_csv(X_test_path, index=False)
y_train.to_csv(y_train_path, index=False)
y_test.to_csv(y_test_path, index=False)

print(f"\nDatos de entrenamiento y prueba guardados en: {os.path.join(os.getcwd(), '..', 'data', 'processed')}")

print("\nPreprocesamiento y división de datos completados. Los datos están listos para el modelado.")
print("--------------------------------------------------")


--------------------------------------------------
5. GUARDADO DE DATOS PREPROCESADOS
--------------------------------------------------

Datos de entrenamiento y prueba guardados en: C:\Users\Víctor\Documents\PaintFormulatorAI\notebooks\..\data\processed

Preprocesamiento y división de datos completados. Los datos están listos para el modelado.
--------------------------------------------------
